## 배민커넥트, 배민커넥터 안전 구글 기사 크롤링

In [70]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 엔터키 입력용
import pandas as pd

driver = webdriver.Chrome('C:/Temp/chromedriver') # 웹드라이버 객체 생성

# 구글을 열어서 배민커넥트까지 입력 후 이동
driver.get('http://www.google.com') # 구글 홈페이지 내용 렌더링
target=driver.find_element_by_css_selector("[name = 'q']") # 검색어 지정
target.send_keys('배달대행') # 타겟에 '파이썬' 입력
target.send_keys(Keys.ENTER) # 엔터처리

# 뉴스 페이지로 이동
news = driver.find_element_by_css_selector('#hdtb-msb-vis > div:nth-child(3) > a')
news.click()

# 뉴스 헤드라인 수집
news_title = []
news_press = []

# 알고리즘을 짜보자
# 다음 페이지 -> #pnnext > span:nth-child(2) // 이걸 19번해야 함 / 마지막에 누르자
for i in range(19):
    
# 한페이지당 뉴스타이틀, 언론사 10개 수집
    for k in range(1,11):
        try : 
            title_address = '#rso > div:nth-child('+ str(k) +') > g-card > div > div > div.dbsr > a > div > div.hI5pFf > div.JheGif.nDgy9d'
            title = driver.find_element_by_css_selector(title_address)
            news_title.append(title.text)
            press_address = '#rso > div:nth-child('+ str(k) +') > g-card > div > div > div.dbsr > a > div > div.hI5pFf > div.XTjFC.WF4CUc'
            press = driver.find_element_by_css_selector(press_address)
            news_press.append(press.text)
        except :
            print(str(i) + "페이지" +str(k) +"번째 기사에서 오류가 발생했습니다.")
    next_btn = driver.find_element_by_css_selector('#pnnext > span:nth-child(2)')
    next_btn.click()
    


#rso > div:nth-child(1) > div > div.yuRUbf > a > h3 > span

0페이지2번째 기사에서 오류가 발생했습니다.
2페이지6번째 기사에서 오류가 발생했습니다.
3페이지2번째 기사에서 오류가 발생했습니다.
4페이지1번째 기사에서 오류가 발생했습니다.
10페이지3번째 기사에서 오류가 발생했습니다.
12페이지6번째 기사에서 오류가 발생했습니다.


In [72]:
import pandas as pd

df = pd.DataFrame(news_title, news_press)
display(df)
df.to_excel('./fusion/data/뉴스_배달대행.xlsx')

,0
매일경제,"스파이더크래프트, 다날과 배달대행시장 결제서비스 혁신 - 매일경제"
논객닷컴,"배달 근로자 실태 현장 목소리....""배달대행 선점업체 횡포 막아야 ..."
IT조선,"네이버, 배달대행 '생각대로'에 400억 투자 추진 - IT조선 > 게임·라이프 >..."
데일리시큐,세종시 신규 배달 대행 플랫폼 '굿보이' 마케팅 지원 서비스 개시
동아일보,스타벅스도 배달 나선다
...,...
한국경제,"배달앱들 ""두 바퀴보다 네 바퀴가 낫다""…경차·초소형 전기차 '배달의 질주'"
매일경제,`자토바이`를 아시나요…판 커진 배달 생태계 `꼼수`도 급증 - 매일경제
조선비즈,"배달의민족, 배달원 1000명 이상 충원… ""배달 품질 향상"""
한국경제,창고형 편의점 떴다…나우픽 vs B마트 '배송전쟁'


## 키워드 추출

In [ ]:
df = pd.read_excel('./fusion/data/뉴스_배달대행.xlsx')

In [58]:
import re

# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거합니다.
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글 처음부터 끝을 알리는 정규표현식
    result = hangul.sub('', text)
    return result

In [59]:
df[0] = df[0].apply(lambda x: text_cleaning(x))
df.tail()

,Unnamed: 0,0
176,매일노동뉴스,지상중계 노동자 건강권포럼 안전보건 새로운 년 여는
177,매일노동뉴스,꺾이고 무너지는 소형 타워크레인
178,매일노동뉴스,산업안전보건법 개정 정치는 무능했고 어머니는 강했다
179,매일노동뉴스,김용균재단 출범 차별없고 안전한 일터 만들겠다
180,매일노동뉴스,감정노동자 보호 산업안전보건법 시행 의미


In [60]:
from konlpy.tag import Okt
from collections import Counter

# 한국어 약식 불용어사전 예시 파일입니다. 출처 - (https://www.ranks.nl/stopwords/korean)
korean_stopwords_path = "./fusion/data/korean_stopwords.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

def get_nouns(x):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    
    # 한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]
    
    # 불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]
    
    return nouns

In [62]:
# ‘ko_text’ 피처에 이를 적용합니다.
# display(df[0])
df['요약'] = df[0].apply(lambda x: get_nouns(x))
print(df.shape)
df.tail()

(181, 2)


,Unnamed: 0,0
176,매일노동뉴스,"[지상, 중계, 노동자, 건강, 포럼, 안전, 보건]"
177,매일노동뉴스,"[소형, 타워, 크레인]"
178,매일노동뉴스,"[산업, 안전, 보건, 개정, 정치, 어머니]"
179,매일노동뉴스,"[김용균, 재단, 출범, 차별, 일터]"
180,매일노동뉴스,"[감정노동자, 보호, 산업, 안전, 보건, 시행, 의미]"


In [63]:
df.to_excel('./fusion/data/뉴스_배민커넥터 안전.xlsx')

## 연관분석

In [44]:
from apyori import apriori

# 장바구니 형태의 데이터(트랜잭션 데이터)를 생성
transactions = [
    ['안전', '현실'],
    ['안전', '소득'],
    ['배민', '커넥터', '후기']
]

# 연관 분석을 수행
results = list(apriori(transactions))
for result in results:
    print(result)

RelationRecord(items=frozenset({'배민'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'배민'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'소득'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소득'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'안전'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'안전'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'커넥터'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'커넥터'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'현실'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'현실'}), confidence=0.3333333333333333, lift=

In [46]:
# 데이터 프레임 형태로 정리
columns = ['source', 'target', 'support']
network_news = pd.DataFrame(columns=columns)

# 규칙의 조건절을 source, 결과절을 target, 지지도를 support 라는 데이터 프레임의 피처로 변환
for result in results:
    if len(result.items) == 2:
        items = [x for x in result.items]
        row = [items[0], items[1], result.support]
        series = pd.Series(row, index=network_news.columns)
        network_news = network_news.append(series, ignore_index=True)

network_news.head()

,source,target,support
0,커넥터,배민,0.333333
1,후기,배민,0.333333
2,안전,소득,0.333333
3,현실,안전,0.333333
4,후기,커넥터,0.333333


In [47]:
# 트랜잭션 데이터를 추출
transactions = tweet['nouns'].tolist()
#transactions = [transaction for transaction in transactions if transaction] # 공백 문자열을 방지합니다.
print(transactions[:10])

NameError: name 'tweet' is not defined

In [ ]:
# 연관 분석을 수행합니다.
results = list(apriori(transactions,
                        min_support=0.1))
print(results) 

## 구글 기사 단어 추출

In [39]:
news_corpus = "".join(df[0].tolist())
print(news_corpus)

먼 곳 배차해주는 이놈, 속을 모르겠네혁신의 '뽕' 배달앱, 법과 규제마저 무력화한다국내 최초 플랫폼 노동 협약은 어떻게 탄생했나배민커넥트, 기자가 직접 해보니… 전동킥보드 5시간 배달, 4만 ...배민커넥터와 배민라이더는 다르다배달 라이더 연봉 1억? 10시간 15건 배달하고 번 돈은…'배민 라이더('배달의 민족' 배달원)' 안전교육은 단 5분, 사고나면 ...배달의민족, 배민커넥트 보험계약 면책 등 주요사항 빠져인천국제공항공사, 지속가능성지수 11년 연속 1위... 대한민국 ...“건물을 뚫고 갈 수는 없잖아요”…'배민라이더'는 내비보다 빠르다?“불법에도 당당한 배달앱… 혁신 아닌 헬조선식 플랫폼”[세모탈] 퇴근한 뒤 배민 커넥트로 출근! 4시간 일하고 번 돈은? - 서울경제자유와 불안 사이를 달리는 새로운 노동비즈한국[경제레시피] 쿠팡 플렉스·배민 커넥트·디버…일자리도 '공유경제 ...배달의민족 배달 알바 ‘배민커넥트’, 직접 해 봤습니다[르포] '배민 커넥트' 쏠쏠하네… 전기자전거 타고 2시간, 수입 3 ...절망의 공유경제... '배달의 민족' 너무한 거 아닙니까배달의민족-DB손보, 자전거·전동 킥보드 라이더 전용 보험 출시장성규도 한 이 알바 직접 해보니, 배달 음식이 달리 보이네3시간 2만6900원···新일자리 '배민 라이더' 최저시급 벅찼다 - 중앙일보라이더유니온 “'우아한형제들' 불공정행위 고발한다”누구나 배달의민족에서 배달하기, ‘배민커넥트’ 가동[배달의 세계] 4시간이면 나도 배달라이더... 걷고 타고 나른다수개월 수입 0…여행사 대표는 오늘도 '배달 콜'만 기다린다 - 중앙일보'도보 배달 서비스'로 더 뜨거워진 배달 시장…"배달원 안전 대책 ...우아한형제들의 전혀 우아하지 않은 '노동 실험'기사 - 배달대행 알바, 배민라이더를 시작했다 3 : 멀고 먼 배달 ...배달앱들 "두 바퀴보다 네 바퀴가 낫다"…경차·초소형 전기차 '배달의 질주'놀 수는 없고… 코로나에 `배달알바` 뛰는 자영업자·휴직자 - 매일경제[해봤습니다-배민커넥트] ⑤'비오

In [40]:
from konlpy.tag import Okt
from collections import Counter

# 명사 키워드를 추출
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(news_corpus)
count = Counter(nouns)

# 한글자 키워드를 제거
remove_char_counter = Counter({x : count[x] for x in count if len(x) > 1})
print(remove_char_counter)

Counter({'배달': 66, '배민': 29, '라이더': 19, '한국': 19, '안전': 18, '민족': 12, '커넥트': 11, '플랫폼': 10, '노동': 9, '코로나': 9, '보험': 8, '노동자': 8, '쿠팡': 7, '비대': 7, '공사': 7, '관리': 7, '관광': 7, '모바일': 6, '서울시': 6, '오페라': 6, '산업': 6, '보건': 6, '보드': 5, '시장': 5, '배송': 5, '체험': 5, '서울': 5, '사업': 5, '직접': 4, '대한민국': 4, '알바': 4, '현장': 4, '정부': 4, '요기': 4, '사고': 4, '업무': 4, '축제': 4, '개최': 4, '예스': 4, '대구': 4, '불안': 3, '일자리': 3, '음식': 3, '중앙': 3, '일보': 3, '형제': 3, '대표': 3, '서비스': 3, '기사': 3, '상품': 3, '사회': 3, '시간': 3, '시간제': 3, '보호': 3, '확산': 3, '화재': 3, '시대': 3, '마켓': 3, '공유': 3, '재난': 3, '제품': 3, '추진': 3, '캠페인': 3, '디지털': 3, '인천': 3, '항공': 3, '드론': 3, '환경': 3, '온라인': 3, '현대': 3, '기업': 3, '처벌': 3, '마케팅': 3, '혁신': 2, '국내': 2, '최초': 2, '협약': 2, '커넥터': 2, '연봉': 2, '안전교육': 2, '계약': 2, '조선': 2, '비즈': 2, '공유경제': 2, '타고': 2, '수입': 2, '자전거': 2, '전동': 2, '시급': 2, '유니온': 2, '불공정': 2, '행위': 2, '고발': 2, '세계': 2, '도보': 2, '대책': 2, '실험': 2, '대행': 2, '바퀴': 2, '질주': 2, '자영': 2, '업자': 2, '매일경제': 2, '퇴근길': 2, '추락': 2, '편의점': 2, 

In [41]:
# 위의 결과를 보기 쉽게 display
# 키워드와 키워드 빈도 점수를 ‘node’, ‘nodesize’ 라는 데이터 프레임의 피처로 생성
node_df = pd.DataFrame(remove_char_counter.items(), columns=['node', 'nodesize'])
node_df = node_df[node_df['nodesize'] >= 5] # 시각화의 편의를 위해 ‘nodesize’ 5 이하는 제거
node_df.head()

,node,nodesize
2,배달,66
7,플랫폼,10
8,노동,9
12,커넥트,11
15,보드,5


In [ ]:
#pip install networkx
import matplotlib.pyplot as plt
import networkx as nx
plt.figure(figsize=(25,25))

# networkx 그래프 객체를 생성합니다.
G = nx.Graph()

# node_df의 키워드 빈도수를 데이터로 하여, 네트워크 그래프의 ‘노드’ 역할을 하는 원을 생성합니다.
for index, row in node_df.iterrows():
    G.add_node(row['node'], nodesize=row['nodesize'])
    
# network_df의 연관 분석 데이터를 기반으로, 네트워크 그래프의 ‘관계’ 역할을 하는 선을 생성합니다.
for index, row in network_df.iterrows():
    G.add_weighted_edges_from([(row['source'], row['target'], row['support'])])
    
# 그래프 디자인과 관련된 파라미터를 설정
pos = nx.spring_layout(G, k=0.6, iterations=50)
sizes = [G.nodes[node]['nodesize']*25 for node in G]
nx.draw(G, pos=pos, node_size=sizes)

# Windows 사용자는 AppleGothic 대신,'Malgun Gothic'. 그 외 OS는 OS에서 한글을 지원하는 기본 폰트를 입력합니다.
nx.draw_networkx_labels(G, pos=pos, font_family='Malgun Gothic', font_size=25)

# 그래프를 출력합니다.
ax = plt.gca()
plt.savefig("./fusion/data/네트워크 결과.png")
plt.show()